In [98]:
from lusidtools.jupyter_tools import toggle_code
from IPython.core.display import HTML

""" Creating portfolios with different tax lot management methods

This notebook demonstrates how to create transaction portfolios under different tax lot accounting methodologies.

Attributes
----------
transactions
holdings
taxlots
accounting
"""

toggle_code("Toggle Docstring")

# Tax Lot Management

Contents:
    1. [Initial Setup](#initialsetup)
    2. [Loading our data](#loading-our-data)
    3. [Cost Basis Comparison](#loading-our-data)


Portfolios in LUSID can be created using any of the following tax lot accounting methods:
- Average Cost (default in LUSID)
- First In First Out (FIFO)
- Last In First Out (LIFO)
- Lowest First
- Highest First

These will determine how tax lots are used to update the cost basis of a transaction portfolio when booking various transactions.
Helpful KB articles:
- [What are the supported tax-lot accounting methods in LUSID?](https://support.lusid.com/knowledgebase/article/KA-01886/en-us)
- [How do I handle different tax lot accounting conventions?](https://support.lusid.com/knowledgebase/article/KA-01887/en-us)

## 1. Initial Setup<a name = "initialsetup"></a>
This section will set up the parameters and methods used in section 2, to compare accounting methods.

In [99]:
# Import LUSID
import lusid
import lusid.models as models
import pandas as pd

# Import Libraries
import pytz
from lusidjam import RefreshingToken
import json
import os
from datetime import datetime
from dateutil.parser import parse

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

pd.set_option('display.float_format', lambda x: f'{x:,.1f}')

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.11051.0


In [100]:
# Create necessary API factories
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
derived_portfolios_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

# 2. Loading our Data<a name = "loading-our-data"></a>

In [101]:
# Specify scope and load data
txns = pd.read_csv("data/taxlot_accounting_transactions.csv",parse_dates=['transaction_date','settlement_date'], date_parser = lambda x : parse(x).strftime('%Y-%m-%d'))
scope = "taxlot_management_examplenb"
txns.drop('instrument_uid', axis=1)

,transaction_id,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal
0,txn_000,FundsIn,"33,000,000",1.0,2020-03-04,2020-03-05,"33,000,000,000",cash_USD
1,txn_001,Buy,"1,000,000",10.0,2020-03-05,2020-03-08,"10,000,000",example_inst
2,txn_002,Buy,"1,000,000",12.0,2020-03-06,2020-03-09,"12,000,000",example_inst
3,txn_003,Buy,"1,000,000",11.0,2020-03-07,2020-03-10,"11,000,000",example_inst
4,txn_004,Sell,"1,500,000",12.5,2020-03-08,2020-03-11,"18,750,000",example_inst


## 2.1 Create Portfolio
When creating a transaction portfolio in LUSID, we pass in a parameter of the `accounting_method` to determine how the tax lots are calculated. If not specified, **Average Cost** is the default method used in LUSID.

In [102]:
def create_upload_portfolio(name, accounting_method):
    try:
        created_date = "2010-01-01T00:00:00.000000+00:00"

        # Create request body
        portfolio_request = models.CreateTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=name,
            base_currency="USD",
            accounting_method=accounting_method, # If not specified - AverageCost is the default accounting method.
            created=created_date,
        )

        # Upload new portfolio to LUSID
        response = transaction_portfolios_api.create_portfolio(
            scope=scope, create_transaction_portfolio_request=portfolio_request
        )

        created = response.version.effective_from
        print(
        f"Portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

In [103]:
def create_derived_portfolio(portfolio_code, parent_code, accounting_method):
    try:
        derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=portfolio_code,
            parent_portfolio_id=models.ResourceId(scope=scope, code= parent_code),
            created="2020-01-01",
            corporate_action_source_id=None,
            accounting_method=accounting_method,
            sub_holding_keys=None,
        )
        response = derived_portfolios_api.create_derived_portfolio(
            scope=scope,
            create_derived_transaction_portfolio_request = derived_portfolio_request)
        created = response.version.effective_from
        print(
        f"Derived portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

In [104]:
# Create a parent portfolio.
portfolio_name_avg = "tax-lot-avgcost"
create_upload_portfolio(portfolio_name_avg, "AverageCost")

Could not create a portfolio with id 'tax-lot-avgcost' because it already exists in scope 'taxlot_management_examplenb'.


In [105]:
# Create portfolios derived from the AverageCost portfolio. (Tax lot type of the parent portfolio does not affect the derived
# portfolios.)
portfolio_name_fifo = "tax-lot-fifo"
create_derived_portfolio(portfolio_name_fifo, portfolio_name_avg, "FirstInFirstOut")

portfolio_name_lifo = "tax-lot-lifo"
create_derived_portfolio(portfolio_name_lifo, portfolio_name_avg, "LastInFirstOut")

portfolio_name_high = "tax-lot-highestcost"
create_derived_portfolio(portfolio_name_high, portfolio_name_avg, "HighestCostFirst")

portfolio_name_low = "tax-lot-lowestcost"
create_derived_portfolio(portfolio_name_low, portfolio_name_avg, "LowestCostFirst")

Could not create a portfolio with id 'tax-lot-fifo' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-lifo' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-highestcost' because it already exists in scope 'taxlot_management_examplenb'.
Could not create a portfolio with id 'tax-lot-lowestcost' because it already exists in scope 'taxlot_management_examplenb'.


## 2.2 Create Instruments

In [106]:
# Create instrument
instr_name=  "Example Instrument"
client_internal = "example_inst"
ticker= "XMPL"

batch_upsert_request = {
    "Example_Equity": models.InstrumentDefinition(
        name=instr_name,
        identifiers={ "ClientInternal": models.InstrumentIdValue(value=client_internal),
                      "Ticker": models.InstrumentIdValue(value=ticker)},
    )
}

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=batch_upsert_request
)

# Check response was successful
if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")

## 2.3 Upload Transactions

In [107]:
def upload_transactions_to_portfolio(portfolio_name, transactions):
    # Upsert transactions
    transactions_request = []
    txn_response = []

    for row, transactions in txns.iterrows():

        if transactions["client_internal"].startswith("cash"):
            instrument_identifier = {"Instrument/default/Currency": "USD"}

        else:
            instrument_identifier = {
                    "Instrument/default/ClientInternal": transactions["client_internal"]
                }

        # Build request body
        transactions_request.append(
            models.TransactionRequest(
                transaction_id=transactions["transaction_id"],
                type=transactions["type"],
                instrument_identifiers=instrument_identifier,
                transaction_date=transactions["transaction_date"].strftime("%Y-%m-%d"),
                settlement_date=transactions["settlement_date"].strftime("%Y-%m-%d"),
                units=transactions["units"],
                transaction_price=models.TransactionPrice(price=transactions["transaction_price"], type="Price"),
                total_consideration=models.CurrencyAndAmount(
                    amount=transactions["total_consideration"], currency="USD"
                ),
            )
        )

        # Make upsert transactions call to LUSID
        txn_response.append(
            transaction_portfolios_api.upsert_transactions(
                scope=scope, code=portfolio_name, transaction_request=transactions_request
            )
        )

    print(f"{len(txn_response)} transactions upserted to portfolio: {portfolio_name}")

In [108]:
# Upload transactions to parent portfolio. The transactions will be automatically added to the derived portfolios.
upload_transactions_to_portfolio(portfolio_name_avg, txns)

5 transactions upserted to portfolio: tax-lot-avgcost


In [109]:
# Show transactions in chronological order
txns.sort_values('transaction_date',ascending=True).drop('instrument_uid', axis=1)

,transaction_id,type,units,transaction_price,transaction_date,settlement_date,total_consideration,client_internal
0,txn_000,FundsIn,"33,000,000",1.0,2020-03-04,2020-03-05,"33,000,000,000",cash_USD
1,txn_001,Buy,"1,000,000",10.0,2020-03-05,2020-03-08,"10,000,000",example_inst
2,txn_002,Buy,"1,000,000",12.0,2020-03-06,2020-03-09,"12,000,000",example_inst
3,txn_003,Buy,"1,000,000",11.0,2020-03-07,2020-03-10,"11,000,000",example_inst
4,txn_004,Sell,"1,500,000",12.5,2020-03-08,2020-03-11,"18,750,000",example_inst


# 3. Cost Basis Comparison<a name = "cost-basis-comp"></a>
Holdings before our sell transaction show the three distinct tax lots and corresponding transactions. We can see that each tax lot has a distinct cost basis based on the transaction price and total consideration of each transaction.

When calling `get_holdings()` in the `display_holding_positions_by_taxlot` method, setting the flag `by_taxlots=true` returns the holdings separated by tax lots, as shown below.

In [110]:
# Prints quick summary from a get_holdings() response of positions for a given effective_at date broken down by tax lots
def display_holding_positions_by_taxlot(effective_at, portfolio_name, show_taxlots = True):
    # Get holdings
    response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_name,
    effective_at = effective_at.isoformat(),
    property_keys=["Instrument/default/Name"],
    by_taxlots=show_taxlots
    )

    # Inspect holdings response for the given effective_at day
    hld = [i for i in response.values]

    names = []
    cost = []
    units = []
    txnid = []
    pchprice = []
    pchdate = []

    for item in hld:
        if item.holding_type_name == "Position":
            names.append(item.properties["Instrument/default/Name"].value.label_value)
            cost.append(item.cost.amount)
            units.append(item.units)
            if show_taxlots:
                txnid.append(item.properties["Holding/default/TaxlotId"].value.label_value)
                pchprice.append(item.properties["Holding/default/TaxlotPurchasePrice"].value.metric_value.value)
                pchdate.append(item.properties["Holding/default/TaxlotPurchaseDate"].value.label_value.strftime("%Y-%m-%d"))

    data = {"cost_basis": cost, "units": units}
    if show_taxlots:
        data = {"transaction_id": txnid, **data , "purchase_price":pchprice, "purchase_date":pchdate}
    summary = pd.DataFrame(data=data, index=names)
    
    date = effective_at.strftime('%Y-%m-%d')
    api_url = api_factory.api_client.configuration._base_path.replace("api","")
    display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope=taxlot_management_examplenb&code={portfolio_name_fifo}&entityType=Portfolio&taxLots=true&effectiveDate={date}" target="_blank">See holdings positions by tax lot in LUSID</a>'))
    display(summary)

In [111]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_name_fifo)

AttributeError: 'str' object has no attribute 'strftime'

We'll now look at the final positions of our portfolios under the 5 accounting methods described above following a sale of 1,500,000 units in a single sell transaction.

## 3.1 Average Cost
Average Cost uses the average price of the final holdings of our portfolios which, in this example, is $ \frac{33,000,000}{3,000,000} = 11.$
When using the **Average Cost** method, our cost basis is averaged across all transactions. Splitting out our holdings by tax lot is thus not applicable. The total cost basis after the sale will be calculated as $11 \times 1,500,000 = 16,500,000$.
**Average Cost** is the default accounting method for a portfolio created in LUSID.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_avg, show_taxlots=False)

## 3.2 First In First Out (FIFO)
Using FIFO, our first tax lot (all 1,000,000 units coming from `txn_001`) will be fully liquidated, while half of our second tax lot (500,000 units coming from `txn_002`) will be liquidated. Below we show the remaining holdings with 500,000 units bought in `txn_002` and all the units bought in `txn_003` in separate tax lots.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_fifo)

## 3.3 Last In Last Out (LIFO)
Using LIFO, our first tax lot (all 1,000,000 units coming from `txn_003`) will be fully sold, while half of our second tax lot (500,000 units coming from `txn_002`) will be sold.
The difference in the total cost basis compared to the **FIFO** method, is due to the difference in total consideration between `txn_001` and `txn_003`.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_lifo)

## 3.4 Highest First
Using Highest First, the equities with the highest transaction price will be sold first. So in the sell transaction of 1,500,000 units, all units from `txn_002` and 500,000 units of `txn_003`. All units from `txn_001` and 500,000 units from `txn_003` remain in the cost basis as shown in the display below.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_high)

## 3.5 Lowest First
Using Lowest First, the equities with the lowest transaction price will be sold first. So in the sell transaction of 1,500,000 units, all units from `txn_001` and 500,000 units of `txn_003`. All units from `txn_002` and 500,000 units from `txn_003` remain in the cost basis as shown in the display below.
The difference in the cost basis compared to the **Highest First** method, is due to the difference in total consideration between `txn_001` and `txn_003`.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_name_low)